In [25]:
import csv
def loadCsv(filename):
    lines = csv.reader(open(filename, "r"))
    dataset = list (lines)
    
    print(dataset[0])
    for i in range(len(dataset)):
        dataset[i]  = [float(x) for x in dataset[i]]
    return dataset

filename = "pima-indians-diabetes.csv"
dataset = loadCsv(filename)
print ("Loaded data file {0} with {1} rows".format(filename, len(dataset)))

['6', '148', '72', '35', '0', '33.6', '0.627', '50', '1']
Loaded data file pima-indians-diabetes.csv with 768 rows


In [2]:
import random
def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset)*splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]


In [3]:
#TEST
dataset = [[1], [2], [3], [4], [5]]
splitRatio = 0.67
train, test = splitDataset(dataset, splitRatio)
print("Split {0} rows into train with {1} and test with{2}".format(len(dataset),train, test))

In [4]:
"""我们将数据特征的获取划分为以下的子任务:
a. 按类别划分数据
b. 计算均值
c. 计算标准差
d. 提取数据集特征
e. 按类别提取属性特征
"""

In [5]:
def seperateByClass(dataset):
    seperated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if(vector[-1] not in seperated):
            seperated[vector[-1]] = []
        seperated[vector[-1]].append(vector)
    return seperated

In [6]:
#TEST
dataset = [[1,20,1], [2,21,0], [3,22,1]]
seperated = seperateByClass(dataset)
print("Seperated instances:{0}".format(seperated))

Seperated instances:{1: [[1, 20, 1], [3, 22, 1]], 0: [[2, 21, 0]]}


In [7]:
import math
#均值
def mean(numbers):
    return sum(numbers)/float(len(numbers))

#标准差
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x - avg, 2) for x in numbers])/float(len(numbers) - 1)
    return math.sqrt(variance)

In [8]:
numbers = [1,2,3,4,5]
print('Summary of {0}: mean={1}, stdev={2}'.format(numbers, mean(numbers), stdev(numbers)))

Summary of [1, 2, 3, 4, 5]: mean=3.0, stdev=1.5811388300841898


In [9]:
#提取数据集的特征
#函数将数据样本按照属性分组为一个个列表，然后可以对每个属性计算均值和标准差。
def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

In [10]:
#TEST
dataset = [[1,20,0], [2,21,1], [3,22,0]]
summary = summarize(dataset)
print('Attribute summaries: {0}'.format(summary))

Attribute summaries: [(2.0, 1.0), (21.0, 1.0)]


In [11]:
#合并代码 先按类别划分 然后计算每个属性的摘要
def summarizeByClass(dataset):
    seperated = seperateByClass(dataset)
    summaries = { }
    for classValue, instances in seperated.items():
        summaries[classValue] = summarize(instances)
    return summaries

In [12]:
#TEST
dataset = [[1,20,1],  [2,21,0], [3,22,1], [4,22,0]]
summary = summarizeByClass(dataset)
print("Summary by class value:{0}".format(summary))

Summary by class value:{1: [(2.0, 1.4142135623730951), (21.0, 1.4142135623730951)], 0: [(3.0, 1.4142135623730951), (21.5, 0.7071067811865476)]}


In [13]:
"""做预测涉及到对于给定的数据样本，计算其归属于每个类的概率，然后选择具有最大概率的类作为预测结果。
    划分成以下任务：
    计算高斯概率密度函数
    计算对应类的概率
    单一预测
    评估精度
"""

'做预测涉及到对于给定的数据样本，计算其归属于每个类的概率，然后选择具有最大概率的类作为预测结果。\n    划分成以下任务：\n    计算高斯概率密度函数\n    计算对应类的概率\n    单一预测\n    评估精度\n'

In [14]:
import math
def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1/(math.sqrt(2*math.pi) * stdev)) * exponent

In [15]:
x= 71.5
mean = 73
stdev = 6.2
probability = calculateProbability(x, mean, stdev)
print("Probabillity of belonging to this class:{0}".format(probability))


Probabillity of belonging to this class:0.06248965759370005


In [16]:
#计算所属类的概率
#给定一个数据样本，它所属每个类别的概率，可以通过将其属性概率相乘得到。结果是一个类值到概率的映射。
def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean,stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    return probabilities

In [17]:
summaries = {0:[(1,0.5)], 1:[(20, 5.0)]}
inputVector  = [1.1, '?']
probabilities = calculateClassProbabilities(summaries, inputVector)
print('Probabilities for each class:{0}'.format(probabilities))

Probabilities for each class:{0: 0.7820853879509118, 1: 6.298736258150442e-05}


In [18]:
 #单一预测
def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel
        

In [19]:
summaries = {'A':[(1, 0.5)], 'B':[(20, 5.0)]}
inputVector = [1.1, '?']
result = predict(summaries, inputVector)
print('Prediction: {0}'.format(result))


Prediction: A


In [20]:
#多重预测
#通过对测试数据集中每个数据样本的预测，我们可以评估模型精度
def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions


In [21]:
summaries = {'A' : [(1, 0.5)], 'B':[(20, 5.0)]}
testSet = [[1.1, '?'], [19.1,'?']]
predictions = getPredictions(summaries, testSet)
print('Predictions : {0}'.format(predictions))

Predictions : ['A', 'B']


In [22]:
#计算精度
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [23]:
testSet = [[1,1,1,'a'], [2,2,2,'a'],[3,3,3,'b']]
predictions = ['a', 'a', 'a']
accuracy = getAccuracy(testSet,predictions)
print('Accuracy: {0}%'.format(accuracy))

Accuracy: 66.66666666666666%


In [24]:
# Example of Naive Bayes implemented from Scratch in Python
import csv
import random
import math
  
def loadCsv(filename):
    lines = csv.reader(open(filename, "r"))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset
  
def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]
  
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated
  
def mean(numbers):
    return sum(numbers)/float(len(numbers))
  
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)
    
def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries
  
def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries
  
def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent
  
def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
    for i in range(len(classSummaries)):
        mean, stdev = classSummaries[i]
        x = inputVector[i]
        probabilities[classValue] *= calculateProbability(x, mean, stdev)
    return probabilities
  
def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel
  
def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions
  
def getAccuracy(testSet, predictions):
    correct = 0
    for i in range(len(testSet)):
        if testSet[i][-1] == predictions[i]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0
  
def main():
    filename = 'pima-indians-diabetes.csv'
    splitRatio = 0.67
    dataset = loadCsv(filename)
    trainingSet, testSet = splitDataset(dataset, splitRatio)
    print('Split {0} rows into train={1} and test={2} rows'.format(len(dataset), len(trainingSet), len(testSet)))
     # prepare model
    summaries = summarizeByClass(trainingSet)
    # test model
    predictions = getPredictions(summaries, testSet)
    accuracy = getAccuracy(testSet, predictions)
    print('Accuracy: {0}%'.format(accuracy))
main()

Split 768 rows into train=514 and test=254 rows
Accuracy: 67.32283464566929%
